# Stock Market Time Series Analysis

This notebook provides a comprehensive analysis of stock market data using:
1. Technical indicators (SMA, EMA, RSI, MACD, Bollinger Bands)
2. Statistical analysis and visualization
3. Time series decomposition and stationarity tests
4. ARIMA modeling

In [1]:
# 1: Setup
import sys
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

scripts_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts'))
if scripts_path not in sys.path:
    sys.path.append(scripts_path)
from scripts.time_series_analysis import TimeSeriesAnalyzer

In [2]:
# 2: Load Multiple DataFrames
import glob

data_dir = os.path.abspath(os.path.join('..', 'data', 'raw', 'historical'))
csv_files = glob.glob(os.path.join(data_dir, '*.csv'))

dataframes = {}
for file in csv_files:
    name = os.path.splitext(os.path.basename(file))[0]
    try:
        df = pd.read_csv(file)
        dataframes[name] = df
        print(f"✅ Loaded {name}: {df.shape}")
    except Exception as e:
        print(f"❌ Failed to load {name}: {e}")

✅ Loaded AAPL_historical_data: (10998, 9)
✅ Loaded AMZN_historical_data: (6846, 9)
✅ Loaded GOOG_historical_data: (5020, 9)
✅ Loaded META_historical_data: (2926, 9)
✅ Loaded MSFT_historical_data: (9672, 9)
✅ Loaded NVDA_historical_data: (6421, 9)
✅ Loaded TSLA_historical_data: (3545, 9)


In [3]:
# 3: Interactive Analysis for Any Dataset
from ipywidgets import Dropdown, interact

def analyze_selected(selected_name):
    df = dataframes[selected_name]
    print(f"\n--- Analyzing {selected_name} ---")
    analyzer = TimeSeriesAnalyzer(
        data=df,
        date_col='Date',
        value_col='Close',
        open_col='Open',
        high_col='High',
        low_col='Low',
        volume_col='Volume'
    )
    analyzer.plot_series(title=f"{selected_name} Close Price Time Series")
    analyzer.plot_decomposition(freq=252)
    stationarity = analyzer.test_stationarity()
    print("ADF Test:", stationarity['ADF'])
    print("KPSS Test:", stationarity['KPSS'])
    analyzer.plot_acf_pacf(lags=40)
    indicators = analyzer.calculate_technical_indicators()
    print("✅ Technical indicators calculated")
    display(indicators.tail())
    model_fit = analyzer.fit_arima(order=(1, 1, 1))
    analyzer.forecast(steps=30)
    analyzer.save_summary_pdf(pdf_path=f"tsa_summary_{selected_name}.pdf")

if dataframes:
    interact(analyze_selected, selected_name=Dropdown(options=list(dataframes.keys()), description='Dataset:'))
else:
    print("No dataframes loaded.")

interactive(children=(Dropdown(description='Dataset:', options=('AAPL_historical_data', 'AMZN_historical_data'…